In [8]:
# =========================
# Fluorescence indices – faceted plot with split Virkisá (English, short 2-line labels)
# Vestari = Red | Virkisá before rain (group_nr==1) = Dark Green | after rain (>1) = Light Green
# =========================

suppressPackageStartupMessages({
  library(tidyverse)
  library(readxl)
  library(tools)
})

# ---- SETTINGS ----
PLOT_MODE <- "box"   # "box" or "violin"
data_dir  <- "C:/Users/sophia/Dropbox/MASTER/Master_Data_Table/FINAL"
stopifnot(dir.exists(data_dir))

# Colors & shapes (print- & colorblind-friendly)
col_vestari   <- "#e41a1c"  # red
col_virk_pre  <- "#1b9e77"  # dark green (before rain)
col_virk_post <- "#b2df8a"  # light green (after rain)
shape_vestari <- 16  # circle
shape_pre     <- 17  # triangle
shape_post    <- 15  # square

# 2-line labels to avoid overlap
labels_grp <- c(
  "Vestari-Jökulsá"       = "Vestari-\nJökulsá",
  "Virkisá (before rain)" = "Virkisá\nbefore\n rain",
  "Virkisá (after rain)"  = "Virkisá\nafter\n rain"
)

# ---- Load data file (xls/xlsx/csv) ----
cands <- list.files(data_dir, pattern = "(?i)\\.(xlsx?|csv)$", full.names = TRUE)
if (length(cands) == 0) stop("No xls/xlsx/csv files found in: ", data_dir)
in_file <- if (length(cands) == 1) cands[1] else cands[which.max(file.info(cands)$mtime)]
message("Reading: ", in_file)
ext <- tolower(file_ext(in_file))

raw <- switch(ext,
  "xls"  = read_excel(in_file),   # if .xls causes issues on your system, save as .xlsx/.csv
  "xlsx" = read_excel(in_file),
  "csv"  = tryCatch(
              readr::read_csv2(in_file, show_col_types = FALSE,
                               locale = readr::locale(decimal_mark = ",", grouping_mark = ".")),
              error = function(e) readr::read_csv(in_file, show_col_types = FALSE)
           ),
  stop("Unexpected extension: ", ext)
)
message("Columns: ", paste(names(raw), collapse = ", "))

# ---- Helpers ----
to_num_safely <- function(x) {
  x_chr <- as.character(x)
  x_chr[x_chr %in% c("#DIV/0!","#NV","#N/V","NaN","NA","","Inf","-Inf","#WERT!")] <- NA
  suppressWarnings(as.numeric(gsub(",", ".", x_chr)))
}

# ---- Required columns (as in your table) ----
needed <- c("river_name","group_nr","bix","fi","hix","E2_E3","S275_295","SR")
missing <- setdiff(needed, names(raw))
if (length(missing) > 0) stop("Missing columns: ", paste(missing, collapse=", "))

dat <- raw %>%
  select(all_of(needed)) %>%
  rename(river = river_name, group = group_nr) %>%
  mutate(
    across(c(bix, fi, hix, E2_E3, S275_295, SR), to_num_safely),
    group = suppressWarnings(as.integer(to_num_safely(group))),
    river = case_when(
      str_detect(tolower(river), "virk") ~ "Virkisá",
      str_detect(tolower(river), "vest") ~ "Vestari-Jökulsá",
      TRUE ~ as.character(river)
    ),
    river_group = case_when(
      river == "Vestari-Jökulsá" ~ "Vestari-Jökulsá",
      river == "Virkisá" & !is.na(group) & group == 1 ~ "Virkisá (before rain)",
      river == "Virkisá" & !is.na(group) & group != 1 ~ "Virkisá (after rain)",
      TRUE ~ NA_character_
    ),
    river_group = factor(river_group,
                         levels = c("Vestari-Jökulsá","Virkisá (before rain)","Virkisá (after rain)"))
  ) %>%
  filter(!is.na(river_group))

# ---- Long format for facets ----
indices_order <- c("hix","bix","fi","E2_E3","S275_295","SR")
plot_df <- dat %>%
  pivot_longer(cols = all_of(indices_order), names_to = "index", values_to = "value") %>%
  filter(is.finite(value)) %>%
  mutate(index = factor(index, levels = indices_order,
                        labels = c("HIX","BIX","FI","E2/E3","S275–295","SR")))

stopifnot(nrow(plot_df) > 0)

# ---- Theme ----
theme_base <- theme_classic(base_size = 11) +
  theme(
    panel.border    = element_rect(color = "black", fill = NA, linewidth = 0.4),
    axis.title.x    = element_blank(),
    axis.title.y    = element_blank(),
    axis.text       = element_text(color = "black"),
    axis.text.x     = element_text(lineheight = 0.95, margin = margin(t = 4)),
    strip.background= element_rect(fill = "white", color = "black", linewidth = 0.4),
    strip.text      = element_text(face = "bold"),
    legend.position = "top",
    legend.title    = element_blank(),
    legend.text     = element_text(lineheight = 0.95),
    plot.caption    = element_text(size = 9, hjust = 0)
  )

scale_fill_grp  <- scale_fill_manual(
  values = c("Vestari-Jökulsá" = col_vestari,
             "Virkisá (before rain)" = col_virk_pre,
             "Virkisá (after rain)"  = col_virk_post),
  labels = labels_grp
)
scale_color_grp <- scale_color_manual(
  values = c("Vestari-Jökulsá" = col_vestari,
             "Virkisá (before rain)" = col_virk_pre,
             "Virkisá (after rain)"  = col_virk_post),
  labels = labels_grp
)
scale_shape_grp <- scale_shape_manual(
  values = c("Vestari-Jökulsá" = shape_vestari,
             "Virkisá (before rain)" = shape_pre,
             "Virkisá (after rain)"  = shape_post),
  labels = labels_grp
)

# ---- Build plot ----
set.seed(123)
base <- ggplot(plot_df, aes(x = river_group, y = value,
                            fill = river_group, color = river_group, shape = river_group))

if (tolower(PLOT_MODE) == "violin") {
  p_fluo <- base +
    geom_violin(width = 0.9, alpha = 0.30, trim = FALSE, linewidth = 0.4) +
    geom_boxplot(width = 0.18, alpha = 0.90, outlier.shape = NA, linewidth = 0.4) +
    geom_jitter(position = position_jitter(width = 0.10, height = 0),
                size = 1.9, alpha = 0.75, stroke = 0.2) +
    stat_summary(fun = mean, geom = "point", shape = 23, size = 2.6,
                 stroke = 0.3, fill = "white") +
    facet_wrap(~ index, nrow = 1, scales = "free_y") +
    scale_x_discrete(labels = labels_grp) +
    scale_fill_grp + scale_color_grp + scale_shape_grp +
    labs(
      title   = "Fluorescence indices by river and rain phase",
      caption = "Violin: density · Box: median & IQR · Diamond: mean · Points: individual samples"
    ) +
    theme_base
} else {
  p_fluo <- base +
    geom_boxplot(width = 0.62, alpha = 0.35, outlier.shape = NA, linewidth = 0.4) +
    geom_jitter(position = position_jitter(width = 0.09, height = 0),
                size = 1.9, alpha = 0.80, stroke = 0.2) +
    stat_summary(fun = mean, geom = "point", shape = 23, size = 2.6,
                 stroke = 0.3, fill = "white") +
    facet_wrap(~ index, nrow = 1, scales = "free_y") +
    scale_x_discrete(labels = labels_grp) +
    scale_fill_grp + scale_color_grp + scale_shape_grp +
    labs(
      title   = "Fluorescence indices by river and rain phase",
      caption = "Box: median & IQR · Diamond: mean · Points: individual samples"
    ) +
    theme_base
}

# ---- Export (PNG, A4, 300 dpi) ----
out_dir <- file.path(data_dir, "Ergebnisse_Fluo")
if (!dir.exists(out_dir)) dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)

ggsave(file.path(out_dir, "Fluorescence_indices_split_A4_portrait.png"),  p_fluo,
       width = 210, height = 297, units = "mm", dpi = 300, bg = "white")
ggsave(file.path(out_dir, "Fluorescence_indices_split_A4_landscape.png"), p_fluo,
       width = 297, height = 210, units = "mm", dpi = 300, bg = "white")

message("Done! Output: ", out_dir, " | Mode: ", PLOT_MODE)


Reading: C:/Users/sophia/Dropbox/MASTER/Master_Data_Table/FINAL/Sophia_Data.xls

Columns: river_name, group_nr, DOC_label, site_nr, day_time, sample, sample_day, sample_date, sample_time, distance_from_glacier, DOC, average_DOC_sample, BDOC_label, BDOC, average_BDOC_sample, BDOC_after_14_days, BDOC_pct, TOC_lab_day_DOC, TOC_lab_day_BDOC, CO2_Flux, Hight_sl, KorExo_Date, KorExo_Time, Time_Fract_Sec, glacier_Name, Chlorophyll_RFU, Chlorophyll_ugperL, Cond_uSpercm, Cond_uSpercm_korr, Depth_m, fDOM_QSU, fDOM_RFU, nLF_Cond_uSpercm, ODO_percentsat, ODO_pctlocal, ODO_mgperL, ORP_mV, Pressure_psi_a, Sal_psu, Sal_psu_korr, SpCond_uSpercm, SpCond_uSpercm_korr, BGA_PC_RFU, BGA_PC_ugperL, TDS_mgperL, TDS_mgperL_korr, Turbidity_FNU, TSS_mgperL, Wiper_Position_volt, pH, pH_mV, Temp_C, Vertical_Position_m, Battery_V, Notizen, Fluo_abs_index_Label_D, bix, b, t, a, m, c, fi, hix, a254, a300, E2_E3, E4_E6, S275_295, S350_400, S300_700, SR, Fluo_abs _index _Label_B, bix_bdoc, b_bdoc, t_bdoc, a_bdoc, m_bd